In [1]:
import pandas as pd
from dateutil.parser import parse

In [10]:
file = open("data/chat_data.txt", encoding = "utf8")

In [11]:
chats = file.readlines()

In [ ]:
chats[:10]

In [13]:
def is_date(string, fuzzy=False):
    try:
        parse(string, fuzzy = fuzzy)
        return True
    except ValueError:
        return False

In [14]:
lines = []

for line in chats:
    line_list = line.replace("\n", "").split(",")
    
    if is_date(line_list[0]):
        lines.append([line_list[0], ("".join(line_list[1:]))])
        
    else:
        lines[-1][-1] = lines[-1][-1] + ' ' + line.replace("\n","")

In [15]:
tidy_format = pd.DataFrame(lines, columns = ['date','message'])

In [ ]:
tidy_format.head()

In [17]:
time_msg = tidy_format["message"].str.split("-", n = 1, expand = True)
tidy_format["time"] = time_msg[0]
tidy_format["message"] = time_msg[1]

In [ ]:
tidy_format.head()

In [18]:
user_msg = tidy_format["message"].str.split(":", n = 1, expand = True)
tidy_format["author"] = user_msg[0]
tidy_format["message"] = user_msg[1]

In [ ]:
tidy_format.head()

In [19]:
tidy_format['id'] = range(1, 1+len(tidy_format))

In [ ]:
tidy_format = tidy_format[["id","date","time","author","message"]]

tidy_format.head()

In [ ]:
pattern = r"(http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+)"

urls_extract = tidy_format.message.str.extractall(pattern)

urls_extract.head()

In [22]:
url_id = []

for i in range(0,len(urls_extract)):
    
    add_list = urls_extract.index[[i]][0][0] + 1
    
    url_id.append(add_list)

In [ ]:
urls_extract['id'] = url_id

urls_extract.head()

In [24]:
links_tbl = pd.merge(tidy_format, urls_extract, on = "id", how = "inner")

In [25]:
links_tbl.columns = ['id', 'date', 'time', 'author', 'message', 'url']

In [ ]:
links_tbl.tail()

In [ ]:
links_tbl.shape

In [ ]:
## links_tbl.to_csv("python_results.csv")